In [1]:
import nltk
import numpy as np
import random
import string
import pandas as pd

from google.cloud import storage
import os
from io import BytesIO

#Connect to GCP bucket and assign the bucket_name and specify the file name
bucket_name = "jupyter-pyspark" #Assign the bucket name where your file is stored
storage_client = storage.Client()

bucket = storage_client.get_bucket(bucket_name)

blob = storage.blob.Blob("emails.csv",bucket)

In [2]:
# Create a pandas dataframe
data = pd.read_csv(BytesIO(blob.download_as_string()))
pd.set_option('display.max_colwidth',-1)
new = data["message"].str.split("\n", n = 15, expand = True) 

data["from"] = new[2]
data["fromn"] = new[8]
data["to"] = new[3]
data["ton"] = new[9]
data["subject"] = new[4]
data["msg"] = new[15]
data.drop(columns =["message"], inplace = True) 
data.drop(columns =["file"], inplace = True) 

data['from'] = data["from"].apply(lambda val: val.replace("From:",''))
data['fromn'] = data["fromn"].apply(lambda val: val.replace("X-From:",''))
data['to'] = data["to"].apply(lambda val: val.replace("To:",''))
data['ton'] = data["ton"].apply(lambda val: val.replace("X-To:",''))
data['subject'] = data["subject"].apply(lambda val: val.replace("Subject:",''))
data['msg'] = data["msg"].apply(lambda val: val.replace("\n",' '))

# Lets look only at emails with 100 words or less and that are Non-replies
new_data=data[(data['msg'].str.len() <100) & ~(data['subject'].str.contains('Re:'))]

/home/mli222/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# Creating an empty Dataframe with column names only
text_body = pd.DataFrame(columns=['email'])
text_body['email'] = data['msg']

In [4]:
text_body.to_csv('data.txt', header=True, index=False, sep=' ', mode='a')